# A single-compartment neuron with Hodgkin & Huxley and transient K+ conductances


## Step 1: Setup

In [ ]:
# Setup inline plotting
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# For Google Colab, this line installs NEURON
!pip install neuron quantities

In [ ]:
# Fetch mechanisms
# Uncomment this line if on google colab
#!git clone https://github.com/ABL-Lab/NSC6084-A25.git

In [ ]:
# Compile the mechanisms
# Note: recompiled mechanisms will not take effect until neuron is imported or the jupyter kernel is restarted

# Uncomment this line if on google colab
#!nrnivmodl ./NSC6084-A25/Sept16/mechanisms
# Uncomment this line if running locally
!nrnivmodl mechanisms

In [ ]:
# We will let this library handle unit conversion for us
import quantities as pq
from quantities import um, nS, mV, cm, ms, nA, S, uF, Hz, degrees, s

In [ ]:
# Import and initialize NEURON
import neuron
from neuron import h
h.load_file("stdrun.hoc")

In [ ]:
# Import other modules we need
import numpy as np

## Step 2: Define the circuit
We will use a single compartment, called a "Section" (more on that in next lectures). <br>
It has a cylindrical geometry with length "L" and a diameter "diam", and a specific capacitance "cm" (capacitance per area) <br>
**Unit conversion is a common source of error, so we will be explicit with our units.** 

In [ ]:
soma = h.Section()

### Query NEURON for the expected units for soma.L & soma.diam

In [ ]:
[h.units(x) for x in ["L", "diam"]]

In [ ]:
soma.L = 10 * um
soma.diam =  10 * um

In [ ]:
volume = soma(0.5).volume() * um**3

In [ ]:
area = soma(0.5).area() * um**2

In [ ]:
area

In [ ]:
volume

### Assign the membrane capacitance "everywhere"

In [ ]:
h.units("cm")  # Query the expected units

In [ ]:
specific_membrane_capacitance = 1 * uF/cm**2

In [ ]:
for sec in soma.wholetree():
    sec.cm = specific_membrane_capacitance #  specific membrane capacitance (micro Farads / cm^2)
    sec.Ra = 100

### Add the Hodgkin-Huxley conductances

In [ ]:
# This model includes the transient Na+, persistent K+ and the leak conductances
soma.insert("hh")

That's almost too easy!

### Parametize the leak conductance G = 1/R

In [ ]:
G = 1 * nS  # R = 1/G in our RC circuit

In [ ]:
v_rest = -70*mV

In [ ]:
tau_m = soma(0.5).cm / soma(0.5).hh.gl

In [ ]:
tau_m = (specific_membrane_capacitance * area / G).rescale(ms)

In [ ]:
tau_m

In [ ]:
# Assign the leak conductance everywhere
for seg in soma:
    seg.hh.gl = (G/area).rescale(S/cm**2)  # Compute specific conductance, and rescale to units of 'S/cm2'
    seg.hh.el = -54.3

### Inspect our parameters

In [ ]:
soma.psection()

In [ ]:
soma.nseg

### Add a current injection

In [ ]:
stim = h.IClamp(soma(0.5))

In [ ]:
stim.delay = 200 * ms  # Inject current 500ms after the start of the simulation 
stim.dur = 600 * ms  # stop injecting current at 520ms 
stim.amp = 0.032 * nA  # Inject 0.1 nA of current

## Step 3: Run the simulation

### Define recordings of simulation variables

In [ ]:
soma_v = h.Vector().record(soma(0.5)._ref_v)
t = h.Vector().record(h._ref_t)

In [ ]:
# Record hh gating variables
hh_vars = ['h', 'm', 'n', 'gna', 'gk']
hh_recordings = {}
for var in hh_vars:
    ref = getattr(soma(0.5).hh, "_ref_"+var )
    hh_recordings[var] = h.Vector().record(ref) 

In [ ]:
hh_recordings

### Run the simulation

In [ ]:
h.finitialize( float(v_rest) )
h.continuerun( float(1000 * ms) )

## Step 4: Plot the results

In [ ]:
plt.plot(t, soma_v, lw=2, label="soma(0.5).v")
plt.legend(fontsize=12)
plt.xlabel("t [ms]", size=16)
plt.ylabel("v [mV]", size=16)
plt.xticks(size=12)
plt.yticks(size=12)
plt.axis([0,1000,-80,30])

In [ ]:
plt.plot(t, hh_recordings['m'], lw=2, label="m")
plt.plot(t, hh_recordings['h'], lw=2, label="h")
plt.plot(t, hh_recordings['n'], lw=2, label="n")
plt.legend(fontsize=12)
plt.xlabel("t [ms]", size=16)
plt.ylabel("fraction", size=16)
plt.xticks(size=12)
plt.yticks(size=12)
plt.axis([195,220,0,1])

In [ ]:
plt.plot(t, hh_recordings['gna'], lw=2, label="gna")
plt.plot(t, hh_recordings['gk'], lw=2, label="gk")
plt.legend(fontsize=12)
plt.xlabel("t [ms]", size=16)
plt.ylabel("fraction", size=16)
plt.xticks(size=12)
plt.yticks(size=12)
plt.axis([195,220,0, 0.05])

In [ ]:
def find_spikes(v, t):
    """ Returns times of spikes for a voltage trace and time grid"""
    
    # look for upward crossing of 0mV
    v_arr = np.array(v)
    t_arr = np.array(t) 
    # This is tricky & powerful notation! Let's discuss in class!
    return t_arr[1:][(v_arr[1:]>0) & (v_arr[:-1]<0)] 

In [ ]:
spike_times = find_spikes(soma_v, t)

In [ ]:
spike_times, len(spike_times)

In [ ]:
firing_freq = (len(spike_times)/(stim.dur*ms)).rescale(Hz)

In [ ]:
firing_freq

In [ ]:
plt.plot(t, soma_v, lw=2, label="soma(0.5).v")
plt.plot(spike_times, len(spike_times)*[0], 'r.')
plt.legend(fontsize=12)
plt.xlabel("t [ms]", size=16)
plt.ylabel("v [mV]", size=16)
plt.xticks(size=12)
plt.yticks(size=12)
plt.axis([200,400,-80,30])

## Now it's your turn!

### **Question 1** 
Create a function to return the firing frequency for a given current input I, and plot the firing frequency for a range of currents from 0 to 0.1 nA (e.g. steps of 0.001 nA)



In [ ]:
I_range = np.arange(0,0.1,0.001)

In [ ]:
def find_freq(I):
    stim.amp = I
    h.finitialize( float(v_rest) )
    h.continuerun( float(1000 * ms) )
    spike_times = find_spikes(soma_v, t)
    firing_freq = (len(spike_times[spike_times>200])/(stim.dur*ms)).rescale(Hz)
    return firing_freq

In [ ]:
# Note this cool notation: List comprehension
freqs = [find_freq(x) for x in I_range]

In [ ]:
plt.plot(I_range, freqs, 'x')
#plt.axis([0.01, 0.02, 0, 100])

In [ ]:
#soma.insert("K_Tst")
#soma(0.5).K_Tst.gK_Tstbar = 0.1*0.477

In [ ]:
soma.insert("Kv4_2_0016")
soma(0.5).Kv4_2_0016.gKv4_2bar = 0.1*0.8
h.celsius = 6.3

In [ ]:
soma(0.5).Kv4_2_0016.q10 = 3.0

In [ ]:
#soma(0.5).K_Tst.gK_Tstbar = 0
#soma(0.5).hh.gkbar = 0.036
#soma(0.5).Kv4_2_0016.gKv4_2bar = 0

In [ ]:
# Note this cool notation: List comprehension
freqs = [find_freq(x) for x in I_range]

In [ ]:
plt.plot(I_range, freqs, 'x')
#plt.axis([0.01, 0.02, 0, 100])

In [ ]:
soma.psection()